In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import joblib
import signal
import sys
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub("[^a-zA-Z]"," ",content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("English")]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}}) 
model = joblib.load('trained_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')
@app.route('/predict', methods=['POST'])
def predict():
    try:  
        data = request.get_json()
        news = data['author']+" "+data["news"]
        news =stemming(news)
        news = vectorizer.transform([news])
        prediction = model.predict(news)
        return jsonify({'prediction': int(prediction[0])})
    except Exception as e:
        return jsonify({'error': e})
signal.signal(signal.SIGINT, signal.SIG_DFL)
if __name__ == '__main__':
    app.run(host='localhost', port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Mar/2024 02:11:52] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 02:11:52] "POST /predict HTTP/1.1" 200 -
